# Instalação das bibliotecas

Primeiramente estamos importando as bibliotecas necessárias para o processamento dos dados, modelagem e avaliação do modelo.


O dataset foi retirado de: [Lung Cancer Data - Keras](https://www.kaggle.com/datasets/wajahat1064/lung-cancer-survey-data?resource=download)

In [6]:
%pip install tensorflow

In [13]:
import numpy as np
import pandas as pd
import gdown
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, f1_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# Preparação dos dados

Em seguida, carregamos o dataset de previsão de câncer e realizamos uma pré-visualização dos dados. Como os dados contêm variáveis categóricas (por exemplo, "GENDER" e "LUNG_CANCER"), essas variáveis precisam ser convertidas em formato numérico para serem utilizadas no modelo. Utilizamos `LabelEncoder` para essa conversão. Também dividimos os dados em conjuntos de treino e teste, e aplicamos normalização nas features para garantir que todas estejam na mesma escala.

In [10]:
arquivo_destino_base = "dataset_{}.csv"

ids = {
    "lung_cancer_survey": "1FiXjDvM1nU5mMuaFL3rZQULi8w4O5aSc",
}

dataframes = {}

for key, file_id in ids.items():
    url = f"https://drive.google.com/uc?id={file_id}"
    arquivo_destino = arquivo_destino_base.format(key)

    gdown.download(url, arquivo_destino, quiet=False)
    df = pd.read_csv(arquivo_destino, sep=";")
    dataframes[key] = df

Downloading...
From: https://drive.google.com/uc?id=1FiXjDvM1nU5mMuaFL3rZQULi8w4O5aSc
To: /content/dataset_lung_cancer_survey.csv
100%|██████████| 11.3k/11.3k [00:00<00:00, 6.68MB/s]


In [11]:
df = pd.read_csv("/content/dataset_lung_cancer_survey.csv", delimiter=",")

In [12]:
df

,GENDER,AGE,SMOKING,YELLOW_FINGERS,ANXIETY,PEER_PRESSURE,CHRONIC DISEASE,FATIGUE,ALLERGY,WHEEZING,ALCOHOL CONSUMING,COUGHING,SHORTNESS OF BREATH,SWALLOWING DIFFICULTY,CHEST PAIN,LUNG_CANCER
0,M,69,1,2,2,1,1,2,1,2,2,2,2,2,2,YES
1,M,74,2,1,1,1,2,2,2,1,1,1,2,2,2,YES
2,F,59,1,1,1,2,1,2,1,2,1,2,2,1,2,NO
3,M,63,2,2,2,1,1,1,1,1,2,1,1,2,2,NO
4,F,63,1,2,1,1,1,1,1,2,1,2,2,1,1,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304,F,56,1,1,1,2,2,2,1,1,2,2,2,2,1,YES
305,M,70,2,1,1,1,1,2,2,2,2,2,2,1,2,YES
306,M,58,2,1,1,1,1,1,2,2,2,2,1,1,2,YES
307,M,67,2,1,2,1,1,2,2,1,2,2,2,1,2,YES


In [16]:
label_encoders = {}
for column in ['GENDER', 'LUNG_CANCER']:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le

df

,GENDER,AGE,SMOKING,YELLOW_FINGERS,ANXIETY,PEER_PRESSURE,CHRONIC DISEASE,FATIGUE,ALLERGY,WHEEZING,ALCOHOL CONSUMING,COUGHING,SHORTNESS OF BREATH,SWALLOWING DIFFICULTY,CHEST PAIN,LUNG_CANCER
0,1,69,1,2,2,1,1,2,1,2,2,2,2,2,2,1
1,1,74,2,1,1,1,2,2,2,1,1,1,2,2,2,1
2,0,59,1,1,1,2,1,2,1,2,1,2,2,1,2,0
3,1,63,2,2,2,1,1,1,1,1,2,1,1,2,2,0
4,0,63,1,2,1,1,1,1,1,2,1,2,2,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304,0,56,1,1,1,2,2,2,1,1,2,2,2,2,1,1
305,1,70,2,1,1,1,1,2,2,2,2,2,2,1,2,1
306,1,58,2,1,1,1,1,1,2,2,2,2,1,1,2,1
307,1,67,2,1,2,1,1,2,2,1,2,2,2,1,2,1


In [18]:
X = df.drop('LUNG_CANCER', axis=1)
y = df['LUNG_CANCER']

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

In [20]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Construção do Modelo em Keras

Aqui, definimos um modelo sequencial simples usando a biblioteca Keras. O modelo contém uma única camada densa (`Dense`) com uma unidade de saída e uma função de ativação sigmoide, como pedido pelo professor. Em seguida, compilamos o modelo utilizando o otimizador Adam, a função de perda `binary_crossentropy` (adequada para problemas de classificação binária) e as métricas `accuracy` e `AUC`. A célula seguinte treina o modelo por 50 épocas com um batch size de 10, utilizando 20% dos dados de treino como validação.

In [21]:
model = Sequential()
model.add(Dense(1, activation='sigmoid', input_dim=X_train.shape[1]))

model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy', 'AUC'])

history = model.fit(X_train, y_train, epochs=50, batch_size=10, validation_split=0.2)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - AUC: 0.5293 - accuracy: 0.5048 - loss: 0.8035 - val_AUC: 0.4444 - val_accuracy: 0.5227 - val_loss: 0.8557
Epoch 2/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - AUC: 0.5033 - accuracy: 0.4945 - loss: 0.8494 - val_AUC: 0.4635 - val_accuracy: 0.5227 - val_loss: 0.8400
Epoch 3/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - AUC: 0.5350 - accuracy: 0.5078 - loss: 0.8417 - val_AUC: 0.4774 - val_accuracy: 0.5227 - val_loss: 0.8261
Epoch 4/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - AUC: 0.5347 - accuracy: 0.4271 - loss: 0.8702 - val_AUC: 0.4809 - val_accuracy: 0.5455 - val_loss: 0.8125
Epoch 5/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - AUC: 0.5184 - accuracy: 0.5691 - loss: 0.7455 - val_AUC: 0.4896 - val_accuracy: 0.5455 - val_loss: 0.8006
Epoch 6/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - AUC: 0.5499 - accuracy: 0.5202 - loss: 0.7623 - val_AUC: 0.5104 - val_accuracy: 0.5455 - val_loss: 0.7878
Epoch 7/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step -

# Avaliação do Modelo

Após o treinamento do modelo, utilizamos o conjunto de teste para fazer previsões. As previsões geradas são comparadas com os rótulos reais para calcular a acurácia e o F1-score.

In [22]:
y_pred = model.predict(X_test)
y_pred_classes = (y_pred > 0.5).astype(int)

accuracy = accuracy_score(y_test, y_pred_classes)
f1 = f1_score(y_test, y_pred_classes)

print(f'Accuracy: {accuracy:.4f}')
print(f'F1 Score: {f1:.4f}')

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Accuracy: 0.8495
F1 Score: 0.9079


A acurácia de 0.8495 significa que o modelo conseguiu prever corretamente cerca de 84,95% das instâncias no conjunto de teste. Esta é uma métrica importante, mas em contextos de classificação binária, já que o modelo pode ter alta acurácia simplesmente por prever a classe majoritária corretamente na maioria das vezes.

O F1-score de 0.9079 indica um equilíbrio muito bom entre precisão (proporção de previsões positivas corretas) e recall (proporção de verdadeiros positivos identificados). Um F1-score alto como este sugere que o modelo está desempenhando bem em ambas as métricas, o que é crucial para detectar corretamente casos de câncer.